In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Librerías para preprocesado de datos
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Librerías para selección de características
from sklearn.feature_selection import (
    SelectKBest, chi2, mutual_info_classif, f_classif, VarianceThreshold,
    RFE, RFECV, SequentialFeatureSelector
)

# Modelos de clasificación y regresión para selección de características y entrenamiento
from sklearn.linear_model import LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.ensemble import RandomForestClassifier

# Librerías para reducción de dimensionalidad
from sklearn.decomposition import PCA

# Librerías para tratamiento de datos desbalanceados
from imblearn.over_sampling import SMOTE

# Librerías para división de los datos
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv("./data/train_data.csv")
df_label = pd.read_csv("./data/train_labels.csv")

In [3]:
# Obtenemos el porcentaje de nulos de cada columna
porcentaje_nulos = df.isnull().mean() * 100

# Nos quedamos con el subconjunto de columnas que tengan porcentaje de nulos superior
# al 50%
porcentaje_nulos = porcentaje_nulos[porcentaje_nulos > 50]

# Eliminamos del Dataframe original las columnas que coincidan con las seleccionadas
# en `porcentaje_nulos`
df = df.drop(columns=porcentaje_nulos.index)

In [4]:
# Seleccionamos las columnas de tipo numérico del DataFrame `df_completo`.
numerical_columns = df.select_dtypes(include=['number']).columns

# Configuramos el imputer para usar la estrategia de imputación de la media.
imputer = SimpleImputer(strategy='mean')

# Aplicamos la transformación de imputación a las columnas numéricas seleccionadas.
# Aquí usamos `fit_transform` para calcular y aplicar la media de cada columna, rellenando los valores nulos con esta media.
df[numerical_columns] = imputer.fit_transform(df[numerical_columns])

In [5]:
columnas_objeto = df.columns[df.dtypes == 'object']

# Creamos la array para el mapeo
frequency_map = {
    'very_low': 0,
    'low': 1,
    'moderate_low': 2,
    'moderate': 3,
    'moderate_high': 4,
    'high': 5,
    'very_high': 6
}
# Crear el imputador con la estrategia de moda
imputer = SimpleImputer(strategy='most_frequent')
for col in ['Infraction_CLH', 'Base_67254', 'Infraction_TEN']:
    df[col] = df[col].map(frequency_map)
    df[col] = imputer.fit_transform(df[[col]])

# Rellenamos valores nulos en 'Infraction_DQLY' con la moda de la columna
# Esto asegura que no queden valores faltantes antes de aplicar One-Hot Encoding
df['Infraction_DQLY'] = df['Infraction_DQLY'].fillna(df['Infraction_DQLY'].mode())

# Definimos las columnas que queremos transformar usando One-Hot Encoding
columns_to_encode = ['Infraction_YFSG', 'Infraction_DQLY']

# Aplicamos pd.get_dummies() a las columnas categóricas seleccionadas
# Esto crea una columna binaria para cada categoría única en estas columnas
df = pd.get_dummies(df, columns=columns_to_encode)

In [6]:
# Seleccionamos las columnas de tipo booleano en el DataFrame
boolean_columns = df.select_dtypes('bool').columns

# Iteramos sobre cada columna booleana para manejar valores nulos y convertir el tipo de datos
for col in boolean_columns:
        # Completamos los valores nulos de la columna con el valor más frecuente
        # Esto se logra utilizando `value_counts().head(1)` para obtener el valor más frecuente (la moda)
        df[col] = df[col].fillna(df[col].value_counts().head(1))

        # Convertimos la columna booleana a tipo entero (0 y 1 en lugar de True y False)
        df[col] = df[col].astype(int)

In [7]:
# Convertimos la columna 'Expenditure_AHF' a formato de fecha y hora (DateTime)
df['Expenditure_AHF'] = pd.to_datetime(df['Expenditure_AHF'])

# Convertimos los datos de fecha a segundos desde el inicio de la época (epoch) y los escalamos a una unidad más manejable
df['Expenditure_AHF'] = df['Expenditure_AHF'].astype(int) / 10**9

In [8]:
df = df.groupby('ID', as_index=False).sum()

In [9]:
# Creamos una instancia del StandardScaler para estandarizar los datos
scaler = StandardScaler()

# Seleccionamos las columnas numéricas del DataFrame
numerical_columns = df.select_dtypes(include=['number']).columns

# Aplicamos el escalador a todas las columnas numéricas
# El método fit_transform calcula la media y desviación estándar de cada columna y transforma los datos
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])


In [10]:
#igualamos La variable X al dataframe base
X = df.drop('ID', axis=1)

# Almacenamos en la variable las etiquestas sin el `ID`
y = df_label.drop('ID', axis=1)

In [ ]:
X = df[['Expenditure_AHF', 'Payment_6804', 'Infraction_CGP', 'Base_7744',
       'Base_80863', 'Risk_1930', 'Expenditure_JIG', 'Infraction_SNZ',
       'Base_02683', 'Infraction_ZWWJ', 'Infraction_QJJF', 'Base_76065',
       'Infraction_EJZ', 'Base_6872', 'Risk_0322', 'Infraction_FMXQ',
       'Infraction_GGO', 'Infraction_TLPJ', 'Base_1165', 'Base_39598',
       'Base_6187', 'Infraction_ZTNC', 'Base_85131', 'Risk_9995',
       'Infraction_AYWV', 'Payment_22507', 'Base_9516', 'Base_36384',
       'Expenditure_FIP', 'Risk_0003', 'Expenditure_HMO', 'Infraction_BSU',
       'Risk_8065', 'Infraction_ZYW', 'Infraction_HSSU', 'Infraction_EHZP',
       'Infraction_TBP', 'Infraction_PBC', 'Infraction_AQO', 'Base_69608',
       'Base_91828', 'Expenditure_IDZ', 'Risk_1475', 'Base_8511',
       'Infraction_JYZB', 'Base_22178', 'Infraction_ZTYG', 'Infraction_EYU',
       'Expenditure_HPM', 'Infraction_HFSI']]
y = df_label['label']

In [31]:
# realizar sobremuestreo en la clase minoritaria. Usamos random_state=42 para asegurar
# que los resultados sean reproducibles en cada ejecución.
smote = SMOTE(random_state=42)

# Aplicamos SMOTE a las características X y las etiquetas y.
# Esto genera nuevas muestras sintéticas de la clase minoritaria, devolviendo un nuevo
# conjunto de datos balanceado en X_resampled e y_resampled.
X_resampled, y_resampled = smote.fit_resample(X, y)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled)

In [33]:
rf = RandomForestClassifier(n_estimators=175, random_state=42, max_depth=22)

rf.fit(X_train, y_train)
y_train_pred = rf.predict(X_train)
f1_score_training = f1_score(y_train, y_train_pred)
print("Training: ", f1_score_training)

y_test_pred = rf.predict(X_test)
f1_score_test = f1_score(y_test, y_test_pred)
print("Test: ", f1_score_test)


Training:  0.9982375921935496
Test:  0.7874559174851162
